In [2]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [3]:
import matplotlib.pyplot as plt

In [4]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [61]:
#Variable 
variable="tg_mean"

files85 = glob.glob(folder+"*"+"rcp85_"+variable+"_annual.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+variable+"_annual.nc")
filesobs = glob.glob(folder+"NRCAN_obs_"+variable+"_annual.nc")

#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_tg_mean_annual.nc']

In [62]:
filesobs

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NRCAN_obs_tg_mean_annual.nc']

# FOR RCP45 

In [6]:
dsEns45= ens.create_ensemble(files45)
#dsEnsSeas45 = ens.create_ensemble(filesSeas)
dsEns45


<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

## Get the percentiles

In [7]:
# Calculate ensembles percentiles on regional means
perctiles45 = ens.ensemble_percentiles(dsEns45)
perctiles45
dsEns45 =0

## Convert in Dataframe

In [8]:
dfper145 = perctiles45.to_dataframe().dropna()
dfper45 = perctiles45.drop('realization').to_dataframe().dropna()
dfmean45 = dfper45.reset_index()
dfper45 = 0

In [9]:
dfmean45

,lat,lon,time,tg_mean_p10,tg_mean_p50,tg_mean_p90
0,66.623306,-89.045212,1950-01-01,257.085693,258.653290,260.071655
1,66.623306,-89.045212,1951-01-01,256.835785,258.389740,259.078644
2,66.623306,-89.045212,1952-01-01,257.146881,258.677917,259.168579
3,66.623306,-89.045212,1953-01-01,257.813049,258.613068,259.164734
4,66.623306,-89.045212,1954-01-01,258.522583,259.148529,259.997162
...,...,...,...,...,...,...
11507101,40.041039,-74.045807,2096-01-01,287.267639,287.622437,289.489594
11507102,40.041039,-74.045807,2097-01-01,286.741150,287.997528,288.674133
11507103,40.041039,-74.045807,2098-01-01,286.326630,287.628967,288.842590
11507104,40.041039,-74.045807,2099-01-01,286.937592,287.839813,290.203461


# FOR RCP85 

In [63]:
dsEns85= ens.create_ensemble(files85)
dsEns85


<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

## Get the percentiles

In [64]:
# Calculate ensembles percentiles on regional means
perctiles85 = ens.ensemble_percentiles(dsEns85)
perctiles85
dsEns85 =0

## Convert in Dataframe

In [65]:
dfper185 = perctiles85.to_dataframe().dropna()
dfper85 = perctiles85.drop('realization').to_dataframe().dropna()
dfmean85 = dfper85.reset_index()
dfper85 = 0

In [66]:
dfmean85

,lat,lon,time,tg_mean_p10,tg_mean_p50,tg_mean_p90
0,66.623306,-89.045212,1950-01-01,257.056732,258.643677,260.095032
1,66.623306,-89.045212,1951-01-01,256.838013,258.383789,259.087128
2,66.623306,-89.045212,1952-01-01,257.158539,258.634918,259.196228
3,66.623306,-89.045212,1953-01-01,257.833679,258.571075,259.167938
4,66.623306,-89.045212,1954-01-01,258.513580,259.165405,260.001831
...,...,...,...,...,...,...
11507101,40.041039,-74.045807,2096-01-01,289.641998,290.565857,292.025970
11507102,40.041039,-74.045807,2097-01-01,289.080017,290.811249,291.779266
11507103,40.041039,-74.045807,2098-01-01,288.290466,290.388062,291.735809
11507104,40.041039,-74.045807,2099-01-01,289.418976,290.153625,291.091431


# FOR OBSERVED DATA

In [75]:
dsOBs= xr.open_mfdataset(filesobs)

will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  dsOBs= xr.open_mfdataset(filesobs)

to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  combined = auto_combine(



In [76]:
dsOBs

<xarray.Dataset>
Dimensions:  (lat: 510, lon: 1068, time: 64)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2013-01-01
  * lon      (lon) float32 -140.95833 -140.875 ... -52.125004 -52.04167
  * lat      (lat) float32 83.45833 83.375 83.291664 ... 41.125 41.041668
Data variables:
    tg_mean  (time, lat, lon) float32 dask.array<chunksize=(64, 510, 1068), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           NRCAN 10km Gridded Climate Dataset
    history:         2012-10-22T13:14:41: Convert from original format to Net...
    institution:     NRCAN
    source:          ANUSPLIN
    redistribution:  Redistribution policy unknown. For internal use only.

## Convert in Dataframe

In [77]:
dfperObs = dsOBs.to_dataframe().dropna()
dfmeanObs = dfperObs.reset_index()
dfperObs = 0

In [78]:
dfmeanObs

,lat,lon,time,tg_mean
0,83.125000,-74.541672,1950-01-01,251.429901
1,83.125000,-74.541672,1951-01-01,252.994797
2,83.125000,-74.541672,1952-01-01,253.964935
3,83.125000,-74.541672,1953-01-01,253.616013
4,83.125000,-74.541672,1954-01-01,253.628616
...,...,...,...,...
16464827,41.708332,-82.208336,2009-01-01,282.584686
16464828,41.708332,-82.208336,2010-01-01,283.870300
16464829,41.708332,-82.208336,2011-01-01,284.144257
16464830,41.708332,-82.208336,2012-01-01,285.570129


## Cut by region

## Get the lat lon regions

In [10]:
regions_folder = "/home/mlopez/EXEC/Grids-polygons-regions/"

In [11]:
files_regions = glob.glob(regions_folder+"*"+"lat_lon.feather")
files_regions

['/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_REG_FOR_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_UA_PER_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/REG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SDOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/Secteurs_Operations_Regionaleslat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SREG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/territoire_guidelat_lon.feather']

In [12]:
listdfreg=[]
for region in files_regions:
    df = pd.read_feather(region)
    listdfreg.append(df)
   

In [13]:
listdfreg[1]

,index,lat,lon,index_right,OBJECTID,PER_NO_UA,DATE_PUBLI,SUPERFICIE,Shape_Leng,Shape_Area,D_GENERAl
0,71387,52.373878,-68.129379,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est
1,71803,52.290550,-68.129379,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est
2,71804,52.290550,-68.046051,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est
3,71805,52.290550,-67.962723,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est
4,71806,52.290550,-67.879387,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est
...,...,...,...,...,...,...,...,...,...,...,...
7611,105464,45.540821,-71.045929,72,73,05151,2019-11-30,2554.5,4.004611e+04,2.559616e+07,None
7612,105881,45.457489,-70.962601,71,72,05151,2019-11-30,2664.9,4.870958e+04,2.671080e+07,None
7613,105884,45.457489,-70.712608,69,70,05151,2019-11-30,9572.5,5.764414e+04,9.594909e+07,None
7614,106297,45.374161,-70.962601,2,3,05151,2019-11-30,1657.5,3.229192e+04,1.662015e+07,None


In [14]:
lisdfregdata45 =[]
for dfregion in listdfreg:
    dfdatareg45 = pd.merge(dfregion, dfmean45, on=["lat","lon"])
    lisdfregdata45.append(dfdatareg45)

In [15]:
listdfreg=[]

In [18]:
dfmeanUA45 = lisdfregdata45[1]

## Get mean by year

In [21]:
dfmean45UA = dfmeanUA45.groupby(["time", "PER_NO_UA"]).mean()

In [31]:
dfmean45UA2 = dfmean45UA.reset_index()

In [38]:
dfmean45UA2

,time,PER_NO_UA,index,lat,lon,index_right,OBJECTID,SUPERFICIE,Shape_Leng,Shape_Area,tg_mean_p10,tg_mean_p50,tg_mean_p90
0,1950-01-01,01171,95201.645161,47.601222,-69.080955,36.145161,37.145161,1.499415e+05,3.953173e+05,1.490932e+09,274.232801,275.068325,276.201072
1,1950-01-01,01272,91460.984375,48.354510,-67.422377,564.085938,565.085938,6.180808e+05,8.312899e+05,6.131052e+09,273.508049,274.342411,275.434308
2,1950-01-01,02371,90404.996016,48.557300,-71.057218,597.888446,598.888446,7.143168e+05,6.101567e+05,7.080669e+09,272.347550,273.084064,274.325299
3,1950-01-01,02471,81598.938190,50.321718,-70.868049,97.000000,98.000000,2.499778e+06,1.670730e+06,2.469022e+10,270.966858,271.482687,272.741235
4,1950-01-01,02571,85527.359649,49.529549,-73.054768,42.996491,43.996491,3.167980e+06,2.073621e+06,3.133969e+10,271.851223,272.346957,273.528776
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8904,2100-01-01,09551,80194.000000,50.618515,-64.473730,476.000000,477.000000,2.518203e+05,5.738118e+05,2.485761e+09,273.486499,276.369704,277.295985
8905,2100-01-01,09751,87291.241877,49.184176,-69.745741,128.000000,129.000000,1.545146e+06,1.160931e+06,1.529547e+10,274.634672,276.877293,277.978032
8906,2100-01-01,11161,90841.280000,48.482036,-66.011465,453.000000,454.000000,6.917589e+05,6.369389e+05,6.859492e+09,275.856706,278.653232,279.252656
8907,2100-01-01,11262,89951.560000,48.662918,-64.905064,126.000000,127.000000,4.403590e+05,5.562723e+05,4.364494e+09,275.895850,278.827626,279.480878


In [39]:
dfmean45UA3 = dfmean45UA2[['time', 'PER_NO_UA', 'tg_mean_p10', 'tg_mean_p50', 'tg_mean_p90']]

In [40]:
dfmean45UA3.PER_NO_UA.unique()

array(['01171', '01272', '02371', '02471', '02571', '02651', '02661',
       '02662', '02663', '02664', '02665', '02666', '02751', '03153',
       '03171', '03351', '03451', '03571', '04151', '04251', '04351',
       '04352', '05151', '06151', '06271', '06452', '06471', '07151',
       '07152', '07251', '07351', '07352', '07451', '08151', '08152',
       '08251', '08351', '08451', '08462', '08551', '08562', '08651',
       '08652', '08663', '08664', '08665', '08666', '08751', '08762',
       '08763', '08764', '09351', '09352', '09471', '09551', '09751',
       '11161', '11262', '11263'], dtype=object)

In [45]:
listdfUA = []
for sr in dfmean45UA3.PER_NO_UA.unique():
    dfUAsr = dfmean45UA3.loc[dfmean45UA3['PER_NO_UA'] == sr]
    listdfUA.append(dfUAsr)
    

In [50]:
dfUA451 = listdfUA[1]

## 4th degre polynomium

In [43]:
def polysmooth(years, y, degree):
    coef = np.polyfit(years, y, degree)
    return np.polyval(coef,years)

In [52]:
years = dfUA451.time.dt.year

In [53]:
rp1 = polysmooth(years,dfUA451.tg_mean_p10,4)
rp5 = polysmooth(years,dfUA451.tg_mean_p50,4)
rp9 = polysmooth(years,dfUA451.tg_mean_p90,4)

## Create DF to plot TimeSeries graph in R

In [54]:
df45 = pd.DataFrame()
df45["time"] = years.values
dfUA451["tg_mean_p10"] = rp1
dfUA451["tg_mean_p50"] = rp5
dfUA451["tg_mean_p90"] = rp9
dfUA451["tg_mean_p10"] = round(dfUA451["tg_mean_p10"]-273.15, 2)
dfUA451["tg_mean_p50"] = round(dfUA451["tg_mean_p50"]-273.15, 2)
dfUA451["tg_mean_p90"] = round(dfUA451["tg_mean_p90"]-273.15, 2)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfUA451["tg_mean_p10"] = rp1

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfUA451["tg_mean_p50"] = rp5

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfUA451["tg_mean_p90"] = rp9

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [55]:
dfUA451

,time,PER_NO_UA,tg_mean_p10,tg_mean_p50,tg_mean_p90
1,1950-01-01,01272,0.22,1.31,2.32
60,1951-01-01,01272,0.23,1.29,2.29
119,1952-01-01,01272,0.24,1.27,2.26
178,1953-01-01,01272,0.25,1.26,2.24
237,1954-01-01,01272,0.26,1.25,2.22
...,...,...,...,...,...
8615,2096-01-01,01272,3.69,5.26,6.53
8674,2097-01-01,01272,3.67,5.27,6.56
8733,2098-01-01,01272,3.66,5.29,6.58
8792,2099-01-01,01272,3.64,5.30,6.60


In [57]:
#plt.plot(years, dfUA451["tg_mean_p50"], markersize='10', color='blue')
#plt.fill_between(years, polysmooth(years, dfUA451.tg_mean_p10,4), polysmooth(years, dfUA451.tg_mean_p90,4), alpha=0.3, color='blue')

## TO DO: 
- Do same for all regions - loop
- Do same for RCP 85 and observed data
- Merge all 3 results for every region to get one csv file 

## Create DF by region

In [25]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [26]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [27]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [ ]:
for region, (name, short) in short_dict.items():
    print(region)
    
    
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall3, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_monthly.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_monthly.json", driver="GeoJSON")

In [ ]:
df45

### TODO:
- This dataframe should include both models (RCP 4.5 nad 8.5) and observations (historic)
- I should add labels and similar colors into the shiny graph

In [ ]:
df45.to_csv("p4tgmean.csv")

In [ ]:
plt.plot(years, df45["tg_mean_p50"], markersize='10', color='blue')
plt.fill_between(years, polysmooth(years, df45.tg_mean_p10,4), polysmooth(years, df45.tg_mean_p90,4), alpha=0.3, color='blue')

In [ ]:
dfmean2.to_csv("timeseries.csv")

### NOTE: this example was for the whole region. 
### TODO: 
- Split DF by regions and get same graph for each region
- Convert degrees in Celsius